### Setting up Copernicus ODYSSEA SST data analysis (online)

This notebook shows how to load ODYSSEA data downloaded from servers at execution time.

In [ ]:
# Import modules
import copernicusmarine
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import inspect
print("signature:", inspect.signature(copernicusmarine.describe))
print(inspect.getdoc(copernicusmarine.describe) or "No docstring available")

[If you have not created a credentials file for copernicus, you can do so by running `copernicusmarine.login()`. Once you provide you username and password, it will create a local config file that copernicus reads when executed, so that you don't have to log in every time you fetch some data]

#### Loading a dataset
We're fetching the most recent ODYSSEA product.

In [ ]:
catalogue = copernicusmarine.describe(contains=["sst_glo_phy_my_l3s"])

In [ ]:
pprint(catalogue)

Let's list the name of the dataset in the product

In [ ]:
odyssea_name = catalogue.products[0].datasets[0].dataset_id

In [ ]:
# Set parameters
lon_min = -85
lon_max = -30
lat_min = 15
lat_max = 60
lon_range = [lon_min, lon_max]
lat_range = [lat_min, lat_max]

data_request = {
   "dataset_id_sst_gap_l3s" : odyssea_name,
   "longitude" : lon_range,
   "latitude" : lat_range,
   "time" : ["2010-01-01", "2010-02-01"]
}

# Load xarray dataset
ds = copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id_sst_gap_l3s"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
)

# Print loaded dataset information
print(ds)

Let's explore the temperature

In [ ]:
ds['adjusted_sea_surface_temperature']

### Example: 1D plot

In [ ]:
center_lat = float(ds.latitude.mean())
center_lon = float(ds.longitude.mean())

sst_center = ds["adjusted_sea_surface_temperature"].sel(
    latitude=center_lat, longitude=center_lon, method="nearest"
)
mask = ~np.isnan(sst_center.values)

plt.plot(sst_center['time'][mask], sst_center.values[mask], '-o')
plt.xlabel('Time (yyyy/mm/dd)')
plt.ylabel('SST (Kelvin)')
plt.title(f"SST at (latitute, longitude) = ({center_lat}, {center_lon})")
plt.xticks(rotation=90)

### Example: Map

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cmaps

In [ ]:
t0 = ds["adjusted_sea_surface_temperature"]["time"][0]
sst_map = ds["adjusted_sea_surface_temperature"].sel(time=t0)

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cplot = plt.contourf(
    sst_map.longitude, sst_map.latitude, (sst_map.values-273.15), 1000,
    cmap = "coolwarm",
    transform = ccrs.PlateCarree()
)

plt.title(f"SST in the NWA on {t0.values}")

cbar = plt.colorbar(cplot)
cbar.ax.set_ylabel('(Celsius)')